# **Очистка и подготовка датасета с вакансиями**

---

## **1. Загрузка файла и датасета**

**Загружаем файл с данными в переменную `ds0` и копируем в `ds`**

In [1]:
!pip install -q openpyxl

In [2]:
import pandas as pd
from numpy import NaN
import json

In [3]:
# настройки вывода
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 60)

In [4]:
# загрузка датасета
ds0 = pd.read_excel('result roles (16.06.2022).xlsx')

In [5]:
# загрузка датасета из Excel занимает почти минуту, а создание копии в пандас - секунду
# если что-то пойдёт не так, можно быстро восстановить исходный датасет из ds0, выполнив эту строку
ds = ds0.copy()

In [6]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76382 entries, 0 to 76381
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                76382 non-null  int64 
 1   id                        76382 non-null  int64 
 2   premium                   76382 non-null  bool  
 3   name                      76382 non-null  object
 4   department                4902 non-null   object
 5   has_test                  76380 non-null  object
 6   response_letter_required  76382 non-null  object
 7   area                      76381 non-null  object
 8   salary                    40874 non-null  object
 9   type                      76379 non-null  object
 10  published_at              76380 non-null  object
 11  created_at                76381 non-null  object
 12  archived                  76380 non-null  object
 13  insider_interview         570 non-null    object
 14  url                   

In [7]:
ds.head(2)

,Unnamed: 0,id,premium,name,department,has_test,response_letter_required,area,salary,type,published_at,created_at,archived,insider_interview,url,alternate_url,relations,employer,snippet,schedule,working_days,working_time_intervals,working_time_modes,accept_temporary,description,experience,key_skills,specializations,region,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50
0,0,49059633,False,Младший специалист ручного тестирования,NaN,False,False,"{'id': '15', 'name': 'Астрахань', 'url': 'http...",NaN,"{'id': 'open', 'name': 'Открытая'}",2022-03-28T09:08:00+0300,2022-03-28T09:08:00+0300,False,NaN,https://api.hh.ru/vacancies/49059633?host=hh.ru,https://hh.ru/vacancy/49059633,[],"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",{'requirement': 'Внимательны и наблюдательны. ...,"{'id': 'remote', 'name': 'Удаленная работа'}",[],[],[],False,<p>В некоторых проектах Яндексу помогают компа...,"{'id': 'noExperience', 'name': 'Нет опыта'}",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Астраханская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,54252278,False,Пентестер,NaN,False,False,"{'id': '15', 'name': 'Астрахань', 'url': 'http...",NaN,"{'id': 'open', 'name': 'Открытая'}",2022-04-01T08:52:06+0300,2022-04-01T08:52:06+0300,False,NaN,https://api.hh.ru/vacancies/54252278?host=hh.ru,https://hh.ru/vacancy/54252278,[],"{'id': '719820', 'name': 'SearchInform', 'url'...",{'requirement': 'Глубокие знания клиентских и ...,"{'id': 'fullDay', 'name': 'Полный день'}",[],[],[],False,<p>Лидер на рынке систем информационной безопа...,"{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","[{'name': 'SQL'}, {'name': 'Linux'}, {'name': ...","[{'id': '1.221', 'name': 'Программирование, Ра...",Астраханская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

## **2. Удаление аномальных записей, дубликатов, ненужных колонок**

**Очистим датасет от "аномальных" записей (из 76382 записей - предположительно 354 шт, примерно 0.46%). Подробно процесс очистки описан в стажировке по разведочному анализу**

In [8]:
# отфильтруем только записи, не выходящие за рамки 28-й колонки (т.е. для которых в колонке 29 - пусто)
ds = ds[ds['Unnamed: 29'].isna()]

# Теперь колонки с 29-й по 50-ю пустые и их тоже можно удалить
ds = ds.drop(ds.columns[29:51], axis=1)

**Самая первая колонка - несквозной индекс - порядковый номер вакансии в датасете в рамках каждого региона. Можем удалить**

In [9]:
# Удалим индекс вакансий по регионам (колонка 'Unnamed: 0')
ds = ds.drop('Unnamed: 0', axis=1)

**Удалим дубликаты записей в датасете**

In [10]:
# подсчёт количества дубликатов
print('Было дубликатов:', ds.duplicated().sum(), end='.   ')
ds = ds.drop_duplicates().reset_index(drop=True)
print('Стало:', ds.duplicated().sum())

Было дубликатов: 34.   Стало: 0


**Удалим ненужные для анализа колонки**

In [11]:
ds = ds.drop(columns=['premium', 'department', 'has_test', 'response_letter_required', 'type', 
                      'published_at', 'created_at', 'archived', 'insider_interview', 'url', 
                      'alternate_url', 'relations', 'snippet', 'working_days', 'working_time_intervals', 
                      'working_time_modes', 'accept_temporary', 'description'])

In [12]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75994 entries, 0 to 75993
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               75994 non-null  int64 
 1   name             75994 non-null  object
 2   area             75994 non-null  object
 3   salary           40608 non-null  object
 4   employer         75994 non-null  object
 5   schedule         75994 non-null  object
 6   experience       75994 non-null  object
 7   key_skills       75994 non-null  object
 8   specializations  75994 non-null  object
 9   region           75994 non-null  object
dtypes: int64(1), object(9)
memory usage: 5.8+ MB


In [13]:
ds.head(2)

,id,name,area,salary,employer,schedule,experience,key_skills,specializations,region
0,49059633,Младший специалист ручного тестирования,"{'id': '15', 'name': 'Астрахань', 'url': 'http...",NaN,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...","{'id': 'remote', 'name': 'Удаленная работа'}","{'id': 'noExperience', 'name': 'Нет опыта'}",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Астраханская область
1,54252278,Пентестер,"{'id': '15', 'name': 'Астрахань', 'url': 'http...",NaN,"{'id': '719820', 'name': 'SearchInform', 'url'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","[{'name': 'SQL'}, {'name': 'Linux'}, {'name': ...","[{'id': '1.221', 'name': 'Программирование, Ра...",Астраханская область


---

## **3. Обработка колонки `area`**

In [14]:
# выведем несколько записей: 1, 74377
ds['area'][1], ds['area'][74377]

("{'id': '15', 'name': 'Астрахань', 'url': 'https://api.hh.ru/areas/15'}",
 "{'id': '68', 'name': 'Омск', 'url': 'https://api.hh.ru/areas/68'}")

In [15]:
# функция возвращает значения area по ключу name
def get_city(area):
    area = area.replace("'", '"')
    return json.loads(area)['name']

In [16]:
# заменим колонку `area` на `city`
ds['area'] = ds['area'].apply(get_city)
ds.rename(columns={'area': 'city'}, inplace=True)

# проверим работу функции, выведем, те же записи: 1, 74377
ds['city'][1], ds['city'][74377]

('Астрахань', 'Омск')

In [17]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75994 entries, 0 to 75993
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               75994 non-null  int64 
 1   name             75994 non-null  object
 2   city             75994 non-null  object
 3   salary           40608 non-null  object
 4   employer         75994 non-null  object
 5   schedule         75994 non-null  object
 6   experience       75994 non-null  object
 7   key_skills       75994 non-null  object
 8   specializations  75994 non-null  object
 9   region           75994 non-null  object
dtypes: int64(1), object(9)
memory usage: 5.8+ MB


In [18]:
ds.head(2)

,id,name,city,salary,employer,schedule,experience,key_skills,specializations,region
0,49059633,Младший специалист ручного тестирования,Астрахань,NaN,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...","{'id': 'remote', 'name': 'Удаленная работа'}","{'id': 'noExperience', 'name': 'Нет опыта'}",[],"[{'id': '1.172', 'name': 'Начальный уровень, М...",Астраханская область
1,54252278,Пентестер,Астрахань,NaN,"{'id': '719820', 'name': 'SearchInform', 'url'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","[{'name': 'SQL'}, {'name': 'Linux'}, {'name': ...","[{'id': '1.221', 'name': 'Программирование, Ра...",Астраханская область


---

## **4. Обработка колонки `key_skills`**

In [19]:
# выведем, например, записи: 1, 2, 4, 12
ds['key_skills'][1], ds['key_skills'][2], ds['key_skills'][4], ds['key_skills'][12]

("[{'name': 'SQL'}, {'name': 'Linux'}, {'name': 'Информационная безопасность'}, {'name': 'Работа с базами данных'}, {'name': 'Базы данных'}]",
 '[]',
 "[{'name': 'Git'}, {'name': 'SQL'}, {'name': 'Тестирование'}, {'name': 'C#'}, {'name': 'API'}]",
 "[{'name': 'Поиск и подбор помещения'}, {'name': 'Оценка недвижимости'}, {'name': 'Коммерческая недвижимость'}]")

In [20]:
# функция возвращает значения key_skills по ключу name, склеенные через символ `^`
def get_skills(skills):
    return skills[1:-1].replace("'}, {'name': '", '^').replace("{'name': '", '').replace("'}", '')

In [21]:
# заменим колонку
ds['key_skills'] = ds['key_skills'].apply(get_skills)

# проверим работу функции, выведем, те же записи: 1, 2, 4, 12
ds['key_skills'][1], ds['key_skills'][2], ds['key_skills'][4], ds['key_skills'][12]

('SQL^Linux^Информационная безопасность^Работа с базами данных^Базы данных',
 '',
 'Git^SQL^Тестирование^C#^API',
 'Поиск и подбор помещения^Оценка недвижимости^Коммерческая недвижимость')

In [22]:
ds.head(2)

,id,name,city,salary,employer,schedule,experience,key_skills,specializations,region
0,49059633,Младший специалист ручного тестирования,Астрахань,NaN,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...","{'id': 'remote', 'name': 'Удаленная работа'}","{'id': 'noExperience', 'name': 'Нет опыта'}",,"[{'id': '1.172', 'name': 'Начальный уровень, М...",Астраханская область
1,54252278,Пентестер,Астрахань,NaN,"{'id': '719820', 'name': 'SearchInform', 'url'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between3And6', 'name': 'От 3 до 6 лет'}",SQL^Linux^Информационная безопасность^Работа с...,"[{'id': '1.221', 'name': 'Программирование, Ра...",Астраханская область


---

## **5. Обработка колонки `specializations`**

**Рассмотрим подробнее структуру записей в колонке:**

In [23]:
# выведем записи 0, 10, 30
ds['specializations'][0], ds['specializations'][10], ds['specializations'][30]

("[{'id': '1.172', 'name': 'Начальный уровень, Мало опыта', 'profarea_id': '1', 'profarea_name': 'Информационные технологии, интернет, телеком'}, {'id': '1.117', 'name': 'Тестирование', 'profarea_id': '1', 'profarea_name': 'Информационные технологии, интернет, телеком'}, {'id': '15.93', 'name': 'Информационные технологии, Интернет, Мультимедиа', 'profarea_id': '15', 'profarea_name': 'Начало карьеры, студенты'}]",
 "[{'id': '17.242', 'name': 'Прямые продажи', 'profarea_id': '17', 'profarea_name': 'Продажи'}, {'id': '17.149', 'name': 'Менеджер по работе с клиентами', 'profarea_id': '17', 'profarea_name': 'Продажи'}, {'id': '17.183', 'name': 'Начальный уровень, Мало опыта', 'profarea_id': '17', 'profarea_name': 'Продажи'}, {'id': '20.20', 'name': 'Агент', 'profarea_id': '20', 'profarea_name': 'Строительство, недвижимость'}, {'id': '12.376', 'name': 'Недвижимость', 'profarea_id': '12', 'profarea_name': 'Консультирование'}]",
 "[{'id': '1.172', 'name': 'Начальный уровень, Мало опыта', 'prof

**Видим, что каждая запись имеет структуру в виде списка словарей, в каждом из которых по ключу `profarea_name` лежит название специализации (сферы деятельности). Почистим записи, оставим только перечисленные через символ `^` сферы деятельности. Выведем для примера первые 10 записей:**

In [24]:
# функция возвращает значения specializations по ключу profarea_name, склеенные через символ `^`
def get_profarea(specializations):
    # разделяем строку по нужному ключу, нулевой элемент списка выкидываем
    spec_list = specializations.split("'profarea_name': '")[1:]
    # удаляем все лишнее из каждого элемента списка
    spec_list = [spec.split("'")[0] for spec in spec_list]
    # удаляем дубликаты, сортируем, объединяем список в строку и возвращаем строку
    return '^'.join(sorted(list(set(spec_list))))

# выведем для примера первые 10 почищенных записей со специализациями (сферами деятельности)
for i in range(10):
    spec_string = ds['specializations'][i]
    print(get_profarea(spec_string))

Информационные технологии, интернет, телеком^Начало карьеры, студенты
Информационные технологии, интернет, телеком
Информационные технологии, интернет, телеком
Информационные технологии, интернет, телеком
Информационные технологии, интернет, телеком
Маркетинг, реклама, PR
Высший менеджмент^Информационные технологии, интернет, телеком
Информационные технологии, интернет, телеком^Консультирование
Консультирование^Продажи^Строительство, недвижимость
Консультирование^Продажи^Строительство, недвижимость


In [25]:
# заменим колонку
ds['specializations'] = ds['specializations'].apply(get_profarea)

# проверим работу функции, выведем, те же записи: 0, 10, 30
ds['specializations'][0], ds['specializations'][10], ds['specializations'][30]

('Информационные технологии, интернет, телеком^Начало карьеры, студенты',
 'Консультирование^Продажи^Строительство, недвижимость',
 'Информационные технологии, интернет, телеком')

**Видим, что очень много сфер деятельности связано с Информационными технологиями, интернетом, телекомом. Объединим и назовем для краткости эту сферу IT**

In [26]:
def get_infotech(specializations):
    return 'IT-сфера' if 'Информационные технологии' in specializations else 'Другая область (не IT)'

ds['it'] = ds['specializations'].apply(get_infotech)

In [27]:
ds.head(2)

,id,name,city,salary,employer,schedule,experience,key_skills,specializations,region,it
0,49059633,Младший специалист ручного тестирования,Астрахань,NaN,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...","{'id': 'remote', 'name': 'Удаленная работа'}","{'id': 'noExperience', 'name': 'Нет опыта'}",,"Информационные технологии, интернет, телеком^Н...",Астраханская область,IT-сфера
1,54252278,Пентестер,Астрахань,NaN,"{'id': '719820', 'name': 'SearchInform', 'url'...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': 'between3And6', 'name': 'От 3 до 6 лет'}",SQL^Linux^Информационная безопасность^Работа с...,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера


---

## **6. Обработка колонки `experience`**

In [28]:
# выведем, например, записи: 0, 4, 1, 171
ds['experience'][0], ds['experience'][4], ds['experience'][1], ds['experience'][171]

("{'id': 'noExperience', 'name': 'Нет опыта'}",
 "{'id': 'between1And3', 'name': 'От 1 года до 3 лет'}",
 "{'id': 'between3And6', 'name': 'От 3 до 6 лет'}",
 "{'id': 'between3And6', 'name': 'От 3 до 6 лет'}")

In [29]:
# функция возвращает значения experience по ключу name
def get_exp(experience):
    experience = experience.replace("'", '"')
    return json.loads(experience)['name']

In [30]:
# заменим колонку
ds['experience'] = ds['experience'].apply(get_exp)

# проверим работу функции, выведем те же записи: 0, 4, 1, 171
ds['experience'][0], ds['experience'][4], ds['experience'][1], ds['experience'][171]

('Нет опыта', 'От 1 года до 3 лет', 'От 3 до 6 лет', 'От 3 до 6 лет')

In [31]:
ds.head(2)

,id,name,city,salary,employer,schedule,experience,key_skills,specializations,region,it
0,49059633,Младший специалист ручного тестирования,Астрахань,NaN,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...","{'id': 'remote', 'name': 'Удаленная работа'}",Нет опыта,,"Информационные технологии, интернет, телеком^Н...",Астраханская область,IT-сфера
1,54252278,Пентестер,Астрахань,NaN,"{'id': '719820', 'name': 'SearchInform', 'url'...","{'id': 'fullDay', 'name': 'Полный день'}",От 3 до 6 лет,SQL^Linux^Информационная безопасность^Работа с...,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера


---

## **7. Обработка колонки `schedule`**

In [32]:
# выведем, например, записи: 0, 1
ds['schedule'][0], ds['schedule'][1]

("{'id': 'remote', 'name': 'Удаленная работа'}",
 "{'id': 'fullDay', 'name': 'Полный день'}")

In [33]:
# функция возвращает значения schedule по ключу name
def get_schedule(schedule):
    schedule = schedule.replace("'", '"')
    return json.loads(schedule)['name']

In [34]:
# заменим колонку
ds['schedule'] = ds['schedule'].apply(get_schedule)

# проверим работу функции, выведем те же записи: 0, 1
ds['schedule'][0], ds['schedule'][1]

('Удаленная работа', 'Полный день')

In [35]:
ds.head(2)

,id,name,city,salary,employer,schedule,experience,key_skills,specializations,region,it
0,49059633,Младший специалист ручного тестирования,Астрахань,NaN,"{'id': '1740', 'name': 'Яндекс', 'url': 'https...",Удаленная работа,Нет опыта,,"Информационные технологии, интернет, телеком^Н...",Астраханская область,IT-сфера
1,54252278,Пентестер,Астрахань,NaN,"{'id': '719820', 'name': 'SearchInform', 'url'...",Полный день,От 3 до 6 лет,SQL^Linux^Информационная безопасность^Работа с...,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера


---

## **8. Обработка колонки `employer`**

In [36]:
# выведем, например, записи: 0, 1
ds['employer'][0], ds['employer'][1]

("{'id': '1740', 'name': 'Яндекс', 'url': 'https://api.hh.ru/employers/1740', 'alternate_url': 'https://hh.ru/employer/1740', 'logo_urls': {'90': 'https://hhcdn.ru/employer-logo/3790847.png', '240': 'https://hhcdn.ru/employer-logo/3790848.png', 'original': 'https://hhcdn.ru/employer-logo-original/837491.png'}, 'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=1740', 'trusted': True}",
 "{'id': '719820', 'name': 'SearchInform', 'url': 'https://api.hh.ru/employers/719820', 'alternate_url': 'https://hh.ru/employer/719820', 'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/940031.png', '240': 'https://hhcdn.ru/employer-logo/4200669.png', '90': 'https://hhcdn.ru/employer-logo/4200668.png'}, 'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=719820', 'trusted': True}")

In [37]:
# функция возвращает значения employer по ключу name
def get_employer(employer):
    employer = employer.replace('"', "'")
    return employer.split("name': '")[1].split("',")[0]

In [38]:
# заменим колонку
ds['employer'] = ds['employer'].apply(get_employer)

# проверим работу функции, выведем те же записи: 0, 1
ds['employer'][0], ds['employer'][1]

('Яндекс', 'SearchInform')

In [39]:
ds.head(6)

,id,name,city,salary,employer,schedule,experience,key_skills,specializations,region,it
0,49059633,Младший специалист ручного тестирования,Астрахань,NaN,Яндекс,Удаленная работа,Нет опыта,,"Информационные технологии, интернет, телеком^Н...",Астраханская область,IT-сфера
1,54252278,Пентестер,Астрахань,NaN,SearchInform,Полный день,От 3 до 6 лет,SQL^Linux^Информационная безопасность^Работа с...,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера
2,54428119,Асессор-тестировщик,Астрахань,NaN,Яндекс,Удаленная работа,Нет опыта,,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера
3,54429041,Тестировщик (асессор),Астрахань,NaN,Яндекс,Удаленная работа,Нет опыта,,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера
4,54653519,Тестировщик.Net,Астрахань,"{'from': 25000, 'to': 50000, 'currency': 'RUR'...",МФК Саммит,Полный день,От 1 года до 3 лет,Git^SQL^Тестирование^C#^API,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера
5,53099198,SMM-копирайтер,Астрахань,"{'from': 45000, 'to': 45000, 'currency': 'RUR'...",Комплекс-А,Полный день,От 3 до 6 лет,Медиапланирование^Интернет-реклама^Реклама^PR^...,"Маркетинг, реклама, PR",Астраханская область,Другая область (не IT)


---

## **9. Обработка колонки `salary`**

In [40]:
# функция преобразования строчного формата записи salary в питоновский словарь. Возвращает словарь
def get_salary_dict(salary):
    salary = salary.replace("'", '"')
    salary = salary.replace('True', 'true')
    salary = salary.replace('False', 'false')
    salary = salary.replace('None', 'false')
    return json.loads(salary)

# проверим работу функции, выведем получившийся словарь, например, для 4-й записи колонки salary
get_salary_dict(ds['salary'][4])

{'from': 25000, 'to': 50000, 'currency': 'RUR', 'gross': True}

**Cоздадим теперь 2 колонки: с минимальной и максимальной зарплатой (`'min_salary'` и `'max_salary'`, соответственно). Учитываем только зарплату в рублях. Все зарплаты приводим к значениям "суммы на руки", (т.е. после вычета налогов): если зарплата указана gross (до вычета налогов), то вычтем из нее 13% НДФЛ**

In [41]:
# функция возвращает минимальную или максимальную зарплату, в зависимости от параметра from_to
def get_salary(salary, from_to):
    if salary is NaN:
        return NaN
    else:
        salary_dct = get_salary_dict(salary)
        if salary_dct['currency'] != 'RUR' or not salary_dct[from_to]:
            return NaN
        result = float(salary_dct[from_to])
        if salary_dct['gross']:
            result = result * 0.87
        return result

# функция возвращает минимальную зарплату (параметр from_to = 'from')
def get_min_salary(salary):
    return get_salary(salary, 'from')

# функция возвращает максимальную зарплату (параметр from_to = 'to')
def get_max_salary(salary):
    return get_salary(salary, 'to')

In [42]:
# новые колонки
ds['min_salary'] = ds['salary'].apply(get_min_salary)
ds['max_salary'] = ds['salary'].apply(get_max_salary)

# колонку salary удалим
ds = ds.drop('salary', axis=1)
ds.shape

(75994, 12)

**Удалим записи с совсем маленькими зарплатами:**

In [43]:
# удалим записи, где min_salary < 5000
ds = ds[(ds['min_salary'] >= 5000) | (ds['min_salary'].isna())]
ds.shape

(75937, 12)

In [44]:
# удалим записи, где max_salary < 5000
ds = ds[(ds['max_salary'] >= 5000) | (ds['max_salary'].isna())]
ds.shape

(75933, 12)

**Итоговый результат:**

In [45]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75933 entries, 0 to 75993
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               75933 non-null  int64  
 1   name             75933 non-null  object 
 2   city             75933 non-null  object 
 3   employer         75933 non-null  object 
 4   schedule         75933 non-null  object 
 5   experience       75933 non-null  object 
 6   key_skills       75933 non-null  object 
 7   specializations  75933 non-null  object 
 8   region           75933 non-null  object 
 9   it               75933 non-null  object 
 10  min_salary       36375 non-null  float64
 11  max_salary       21183 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 7.5+ MB


In [46]:
ds.head(6)

,id,name,city,employer,schedule,experience,key_skills,specializations,region,it,min_salary,max_salary
0,49059633,Младший специалист ручного тестирования,Астрахань,Яндекс,Удаленная работа,Нет опыта,,"Информационные технологии, интернет, телеком^Н...",Астраханская область,IT-сфера,NaN,NaN
1,54252278,Пентестер,Астрахань,SearchInform,Полный день,От 3 до 6 лет,SQL^Linux^Информационная безопасность^Работа с...,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера,NaN,NaN
2,54428119,Асессор-тестировщик,Астрахань,Яндекс,Удаленная работа,Нет опыта,,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера,NaN,NaN
3,54429041,Тестировщик (асессор),Астрахань,Яндекс,Удаленная работа,Нет опыта,,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера,NaN,NaN
4,54653519,Тестировщик.Net,Астрахань,МФК Саммит,Полный день,От 1 года до 3 лет,Git^SQL^Тестирование^C#^API,"Информационные технологии, интернет, телеком",Астраханская область,IT-сфера,21750.0,43500.0
5,53099198,SMM-копирайтер,Астрахань,Комплекс-А,Полный день,От 3 до 6 лет,Медиапланирование^Интернет-реклама^Реклама^PR^...,"Маркетинг, реклама, PR",Астраханская область,Другая область (не IT),45000.0,45000.0


---

## **10. Запись итогового датасета в файл**

In [47]:
# запись в файл
ds.to_excel('vacancies.xlsx')